# Load dataset

In [2]:
with open("data_path.txt", "r") as f:
    data_path = f.read()

In [3]:
from tensorflow.keras.utils import image_dataset_from_directory

2022-09-15 09:49:06.857758: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-15 09:49:07.446227: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-15 09:49:07.446274: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-15 09:49:07.529539: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-15 09:49:09.012928: W tensorflow/stream_executor/platform/de

In [4]:
batch_size = 32
size = (224, 224)

In [8]:
train_ds = image_dataset_from_directory(
  data_path,
  validation_split=0.9,
  subset="training",
  seed=123,
  image_size=size,
  batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 2500 files for training.


In [9]:
val_ds = image_dataset_from_directory(
  data_path,
  validation_split=0.05,
  subset="validation",
  seed=123,
  image_size=size,
  batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 1250 files for validation.


In [14]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 224, 224, 3)
(32,)


2022-09-15 09:51:56.983373: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19267584 exceeds 10% of free system memory.


# CNN

In [10]:
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [15]:
num_classes = len(train_ds.class_names)

model = keras.Sequential([
  keras.Input(shape=(224, 224, 3)),
  keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(num_classes-1)])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 112, 112, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 28, 28, 64)      

2022-09-15 09:55:05.575870: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 25690112 exceeds 10% of free system memory.


In [16]:
epochs = 10
callback = EarlyStopping(monitor='loss', patience=3)
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/10
79/79 [==============================] - 67s 836ms/step - loss: 21.9877 - binary_accuracy: 0.5368 - val_loss: 0.6767 - val_binary_accuracy: 0.5816
Epoch 2/10
79/79 [==============================] - 70s 864ms/step - loss: 0.5237 - binary_accuracy: 0.7252 - val_loss: 0.6815 - val_binary_accuracy: 0.6504
Epoch 3/10
79/79 [==============================] - 70s 869ms/step - loss: 0.3421 - binary_accuracy: 0.8420 - val_loss: 0.7715 - val_binary_accuracy: 0.6328
Epoch 4/10
79/79 [==============================] - 84s 1s/step - loss: 0.2222 - binary_accuracy: 0.9100 - val_loss: 0.9360 - val_binary_accuracy: 0.6272
Epoch 5/10
79/79 [==============================] - 76s 945ms/step - loss: 0.1426 - binary_accuracy: 0.9496 - val_loss: 1.1615 - val_binary_accuracy: 0.6384
Epoch 6/10
79/79 [==============================] - 75s 919ms/step - loss: 0.0956 - binary_accuracy: 0.9708 - val_loss: 1.2134 - val_binary_accuracy: 0.6392
Epoch 7/10
79/79 [==============================] - 67s 832m

KeyboardInterrupt: 

## Performances

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Transfert learning

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)
model.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
x = model(inputs)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

model.summary()

In [ ]:
epochs = 10
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

# References

* https://github.com/PacktPublishing/Learning-Computer-Vision-with-TensorFlow
* https://www.tensorflow.org/tutorials/images/classification
* https://keras.io/guides/transfer_learning/